In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5


In [3]:
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300572  [    0/60000]
loss: 2.299186  [ 6400/60000]
loss: 2.288161  [12800/60000]
loss: 2.284814  [19200/60000]
loss: 2.275703  [25600/60000]
loss: 2.252310  [32000/60000]
loss: 2.269259  [38400/60000]
loss: 2.243497  [44800/60000]
loss: 2.256830  [51200/60000]
loss: 2.233360  [57600/60000]
Test Error: 
 Accuracy: 32.3%, Avg loss: 0.035154 

Epoch 2
-------------------------------
loss: 2.252078  [    0/60000]
loss: 2.263878  [ 6400/60000]
loss: 2.225819  [12800/60000]
loss: 2.233094  [19200/60000]
loss: 2.199765  [25600/60000]
loss: 2.159687  [32000/60000]
loss: 2.207577  [38400/60000]
loss: 2.151108  [44800/60000]
loss: 2.192027  [51200/60000]
loss: 2.132020  [57600/60000]
Test Error: 
 Accuracy: 34.0%, Avg loss: 0.033746 

Epoch 3
-------------------------------
loss: 2.183842  [    0/60000]
loss: 2.199530  [ 6400/60000]
loss: 2.125559  [12800/60000]
loss: 2.138086  [19200/60000]
loss: 2.060247  [25600/60000]
loss: 2.007696  [32000/600